In [1]:
#importamos las librerias
import cv2
import numpy as np

In [2]:
#cargamos el modelo COCO 80 clases
rcnn = cv2.dnn.readNetFromTensorflow('DNN/frozen_inference_graph_coco.pb',
'DNN/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt')

In [3]:
#leemos la imagen
img = cv2.imread('img1.jpeg')
alto, ancho, _ = img.shape
#generamos los colores
colores = np.random.randint(0,255,(80,3))


In [5]:
#preparar nuestra imagen
blob = cv2.dnn.blobFromImage(img, swapRB = True)

In [6]:
#procesamos la imagen
rcnn.setInput(blob)

In [7]:
#extraemos los rect y mascaras
info, masks = rcnn.forward(["detection_out_final","detection_masks"])

In [9]:
#extraemos la cantidad de objetos seleccionados
contObject = info.shape[2]

In [10]:
#iteramos la cantidad de objetos detectados
for i in range(contObject):
    #extraemos los rectangulos de los objetos
    inf = info[0,0,1]
    #extramos la clase
    clase = int(inf[1])
    #extraemos puntaje
    puntaje = inf[2]
    #filtramos los objetos con una puntuación alta (mejor identificados)
    if puntaje < 0.70:
        continue
    #coordenadas de rectangulos para detección de objetos
    x = int(inf[3] * ancho)
    y = int(inf[4] * alto)
    x2 = int(inf[5] * ancho)
    y2 = int(inf[6] * alto)
    #extraemos el tamaño de los objetos
    tamobj = img[y:y2, x:x2]
    tamalto, tamancho, _ = tamobj.shape
    #extraemos mascara
    mask = masks[i, clase]
    mask = cv2.resize(mask, (tamancho, tamalto))
    #establecemos un umbral
    _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY)
    mask = np.array(mask, np.uint8)
    #extraemos coordenadas de la mascara
    contornos, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #elegimos los colores
    color = colores[clase]
    r = int(color[0])
    g = int(color[1])
    b = int(color[2])
    #iteramos los contornos
    for cont in contornos:
        #dibujamos mascara
        cv2.fillPoly(tamobj, [cont], (r,g,b))
        #dibujamos
        cv2.rectangle(img,(x,y),(x2,y2),(r,g,b), 3)
    
    #mostramos
    cv2.imshow('TAMANO OBJETO', tamobj)
    # # mascara
    # cv2.imshow('MASCARA', mask)
    cv2.waitKey(0)


cv2.imshow('MASK', img)
cv2.waitKey(0)






KeyboardInterrupt: 

In [ ]:
cv2.imshow('MASK', img)
cv2.waitKey(0)